In [ ]:
!pip install simfin

In [ ]:
import simfin as sf
import pandas as pd

In [ ]:
sf.set_api_key('d409b92f-778c-4032-86f3-ea08133424c4')    #Get SimFin API Key
sf.set_data_dir('~simfin_data/')                          #Set Directory

In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns

In [ ]:
income_df = sf.load_income(variant = 'quarterly', market = 'us').reset_index()    #Load income statements
income_df

In [ ]:
#Calculate Basic and Diluted EPS columns columns using income statements and add columns to dataframe

income_df['Basic EPS'] = income_df['Net Income (Common)'] / income_df['Shares (Basic)']
income_df['Diluted EPS'] = income_df['Net Income (Common)'] / income_df['Shares (Diluted)']
income_df

In [ ]:
#Load cashflow statements

cashflow_df = sf.load_cashflow(variant = 'quarterly', market = 'us').reset_index()
cashflow_df

In [ ]:
#Drop columns that will end up as duplicates when merging income stmt, cashflow stmt, balance sheet

cashflow_df = cashflow_df.drop(columns = ['Report Date',
                                          'Currency',
                                          'Publish Date',
                                          'Restated Date',
                                          'Shares (Basic)',
                                          'Shares (Diluted)',
                                          'Depreciation & Amortization'])
cashflow_df

In [ ]:
balance_df = sf.load_balance(variant = 'quarterly', market = 'us').reset_index()
balance_df

In [ ]:
#Load company info

companies_info_df = sf.load_companies(market = 'us').reset_index()
companies_info_df

In [ ]:
#Prepare to merge with sector and industry data
#Select all rows and only first 4 columns

companies_info_df = companies_info_df.iloc[:, :4]
companies_info_df

In [ ]:
#View bottom of dataset to see how many NaN rows to cut off

companies_info_df.tail(75)

In [ ]:
#Drop off messy rows containing unusable data

companies_info_df.drop(companies_info_df.index[-74:], inplace=True)

In [ ]:
companies_info_df

In [ ]:
sector_industry = sf.load_industries().reset_index()
sector_industry

In [ ]:
#Select the three columns in the dataset to merge with company info df

sector_industry = sector_industry[['IndustryId', 'Sector', 'Industry']]
sector_industry

In [ ]:
companies_industries_df = pd.merge(companies_info_df, sector_industry, on = 'IndustryId', how = 'left')
companies_industries_df

In [ ]:
#Drop overlapping columns with income stmt, balance sheet, and cashflow stmt in preparation to merge all

balance_df = balance_df.drop(columns = ['Report Date',
                                        'Currency',
                                        'Publish Date',
                                        'Restated Date',
                                        'Shares (Basic)',
                                        'Shares (Diluted)'])
balance_df

In [ ]:
merged_df = companies_industries_df.merge(income_df, on = ['Ticker', 'SimFinId'], how = 'left')
                                   
merged_df

In [ ]:
merged_df_2 = pd.merge(
    merged_df, 
    cashflow_df, 
    on=['Ticker', 'SimFinId', 'Fiscal Year', 'Fiscal Period'], 
    how='left'
)


merged_df_3 = pd.merge(
    merged_df_2, 
    balance_df, 
    on=['Ticker', 'SimFinId', 'Fiscal Year', 'Fiscal Period'], 
    how='left'
)

merged_df_3

In [ ]:
merged_df_3 = merged_df_3.drop(columns = ['SimFinId','IndustryId'])

In [ ]:
merged_df_3.shape

In [ ]:
# unique ticker symbols in dataset
len(merged_df_3['Ticker'].unique().tolist())

In [ ]:
#Pull single company out of dataframe to use in analysis

mastercard_df = merged_df_3[merged_df_3['Ticker'] == 'MA'].reset_index(drop = True)
mastercard_df

In [ ]:
#Get Columns containing NaN

mastercard_df.isnull()

In [ ]:
#Get column NaN percentages and prepare to drop or replace values

pct_missing = mastercard_df.isnull().sum() * 100 / len(mastercard_df)
pct_missing_df = pd.DataFrame({'% Missing': pct_missing}).reset_index()
pct_missing_df = pct_missing_df.sort_values(by = '% Missing', ascending=False)
pct_missing_df

In [ ]:
#Drop columns containing mostly missing data - those that won't be useful in analysis

drop_columns = pct_missing_df[ pct_missing_df['% Missing'] > 60]['index'].tolist()
drop_columns

In [ ]:
mastercard_df = mastercard_df.drop(columns = drop_columns)
mastercard_df

In [ ]:
mastercard_df.isnull().sum()

In [ ]:
#after dropping all columns that wont be useful, there are still two columns, ST Debt, and Net acquisitions and divestitures
#Values can be replaced by zero, after confirming that debt and acquisitions and divestitures in those years was in fact, 0.
#If it turned out to be an issue in the dataset, and those values were not supposed to be 0 - mean, median, or other values can be used

mastercard_df.fillna(0, inplace = True)
mastercard_df.isnull().sum()

In [ ]:
#Perform one hot encoding to Fiscal Period data to prepare for Machine Learning

fiscal_encoded = pd.get_dummies(mastercard_df['Fiscal Period'], dtype = int)
fiscal_encoded

In [ ]:
#Drop fiscal period column and replace it with the one hot encoded version

mastercard_df = mastercard_df.drop('Fiscal Period', axis = 1)
mastercard_df = pd.concat([mastercard_df, fiscal_encoded],axis = 1)
mastercard_df

In [ ]:
#confirm Report Date is datetime64
mastercard_df.info()

In [ ]:
#Because the one hot encoded values does not require the int64 datatype, uint8 will take up less memory
#convert to uint8

mastercard_df[['Q1', 'Q2', 'Q3', 'Q4']] = mastercard_df[['Q1', 'Q2', 'Q3', 'Q4']].astype('uint8')
mastercard_df.info()

In [ ]:
#Create a new concatenated column calculating revenue QoQ percent change

mastercard_df['Revenue' + ' change from last quarter'] = mastercard_df['Revenue'].pct_change()
mastercard_df

In [ ]:
#Same concatenation as previous block of code, now performing QoQ changes to desired variables

features_get_delta = ['Shares (Basic)',
                      'Shares (Diluted)',
                      'Operating Income (Loss)',
                      'Net Income',
                      'Total Liabilities & Equity']

for feature in features_get_delta:
    mastercard_df[feature + ' change from last quarter'] = mastercard_df[feature].pct_change()

In [ ]:
mastercard_df

In [ ]:
# Create target (output) label that ML model will be trained to predict
#shift the Basic EPS column by one sample and place results in new column
# target label is the quarterly pct change in EPS
mastercard_df['Next Quarter EPS'] = mastercard_df['Basic EPS'].shift(-1)
mastercard_df

In [ ]:
mastercard_df = mastercard_df.dropna(subset = ['Next Quarter EPS']).reset_index(drop = True)
mastercard_df

In [ ]:
mastercard_df = mastercard_df.dropna().reset_index(drop = True)
mastercard_df

In [ ]:
#Define function to return the target variable to be used in regression

def regression_label(next_quarter, current_quarter):
    if abs(current_quarter) == 0:
        return 0
    return(next_quarter - current_quarter) / abs(current_quarter)

mastercard_df['% Quarterly change in EPS (Regression Label)'] =\
mastercard_df.apply(lambda row: regression_label(row['Next Quarter EPS'], row['Basic EPS']), axis = 1)

mastercard_df